In [1]:
import os
import seaborn as sns
import matplotlib.pyplot as plt
import io
import time
import datetime
import time
import pandas as pd
import numpy as np

In [4]:
os.chdir('C:\\Users\\heyup\\Documents\\DSO562\\Application')

In [7]:
mydata = pd.read_csv('applications data 4.0.csv')

In [8]:
mydata.head()

,record,date,ssn,firstname,lastname,address,zip5,dob,homephone,fraud_label,...,"ssn,namedob,homephone7","ssn,namedob,homephone14","ssn,namedob,homephone31","fulladdress,namedob,homephone1","fulladdress,namedob,homephone2","fulladdress,namedob,homephone3","fulladdress,namedob,homephone7","fulladdress,namedob,homephone14","fulladdress,namedob,homephone31",wday_risk
0,755034,2016-10-03,94616828,EEZESZMUT,SXZXJRJT,0 ERRAZ PL,34664,19760625,1.725564e+09,0,...,0,0,0,0,0,0,0,0,0,0.013480
1,309451,2016-04-23,429131124,SXXSZJMM,SJUMSETR,0 ETEJX AVE,31011,19070626,1.865228e+09,0,...,0,0,0,0,0,0,0,0,0,0.014968
2,699802,2016-09-13,29986017,MJRMJMUSU,RMJUMURZ,0 ETUJX PL,23803,19681112,9.635909e+08,0,...,0,0,0,0,0,0,0,0,0,0.014070
3,834948,2016-11-01,527515589,RUUUXXJXS,SJASXXZM,0 EUSUM WY,24200,19540403,7.645068e+09,0,...,0,0,0,0,0,0,0,0,0,0.014070
4,737752,2016-09-27,483656254,SZEXUMTMM,EEZUURTA,0 RAAZJ ST,721,19170829,4.246577e+09,0,...,0,0,0,0,0,0,0,0,0,0.014070


# Exclude Out-of-Time dataset

In [24]:
tdata = mydata[mydata['date'] <="2016-10-31"]

In [25]:
# Drop original fields, except Fraud label
tdata.drop(['record','date','ssn','firstname','lastname','address','zip5','dob','homephone'], axis =1, inplace =True)

# Generate a column of random values
tdata['Random'] = np.random.ranf(len(tdata))

C:\Users\heyup\Anaconda3\lib\site-packages\pandas\core\frame.py:3694: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  errors=errors)
C:\Users\heyup\Anaconda3\lib\site-packages\ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """


# Perform KS Test

In [38]:
goods = tdata[tdata['fraud_label']==0]
bads = tdata[tdata['fraud_label'] == 1]

numgoods = len(goods)
numbads = len(bads)

In [32]:
KS = pd.DataFrame(np.zeros((goods.shape[1],3)))
i = 0
for column in tdata:
    print(i,column)
    KS.loc[i,0] = column
    i = i+1


0 fraud_label
1 ssn_days
2 fulladdress_days
3 namedob_days
4 homephone_days
5 ssn,fulladdress_days
6 ssn,namedob_days
7 ssn,homephone_days
8 fulladdress,namedob_days
9 fulladdress,homephone_days
10 namedob,homephone_days
11 ssn,fulladdress,namedob_days
12 ssn,fulladdress,homephone_days
13 ssn,namedob,homephone_days
14 fulladdress,namedob,homephone_days
15 ssn1
16 ssn2
17 ssn3
18 ssn7
19 ssn14
20 ssn31
21 fulladdress1
22 fulladdress2
23 fulladdress3
24 fulladdress7
25 fulladdress14
26 fulladdress31
27 namedob1
28 namedob2
29 namedob3
30 namedob7
31 namedob14
32 namedob31
33 homephone1
34 homephone2
35 homephone3
36 homephone7
37 homephone14
38 homephone31
39 ssn,fulladdress1
40 ssn,fulladdress2
41 ssn,fulladdress3
42 ssn,fulladdress7
43 ssn,fulladdress14
44 ssn,fulladdress31
45 ssn,namedob1
46 ssn,namedob2
47 ssn,namedob3
48 ssn,namedob7
49 ssn,namedob14
50 ssn,namedob31
51 ssn,homephone1
52 ssn,homephone2
53 ssn,homephone3
54 ssn,homephone7
55 ssn,homephone14
56 ssn,homephone31
57 full

In [35]:
import scipy.stats as sps

KS.columns = ['field','ks','FDR']

i =0
for column in tdata:
    KS['ks'][i] = sps.ks_2samp(goods[column],bads[column])[0]
    i = i+1

KS.sort_values(by=['ks'],ascending = False, inplace = True)

C:\Users\heyup\Anaconda3\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until


# FDR @ 3%

In [39]:
topRows = int(round(len(tdata)*0.03))
j=0
for column in tdata:
    temp1 = tdata.sort_values(column, ascending = False).head(topRows)
    temp2 = tdata.sort_values(column, ascending = True).head(topRows)
    needed1 = temp1.loc[:,'fraud_label']
    needed2 = temp2.loc[:,'fraud_label']
    FDR1 = sum(needed1)/numbads
    FDR2 = sum(needed2)/numbads
    FDRate = np.maximum(FDR1,FDR2)
    KS.loc[j, 'FDR'] = FDRate
    j=j+1
    

# Combined Score: KS + FDR 3%

In [43]:
KS['Combined_Score'] = KS.apply(lambda x: x['ks'] + x['FDR'], axis=1)
KS.sort_values(by=['Combined_Score'],ascending = False, inplace = True)

In [44]:
KS

,field,ks,FDR,Combined_Score
0,fraud_label,1.000000,1.000000,2.000000
26,fulladdress31,0.335294,0.361123,0.696417
25,fulladdress14,0.326352,0.349796,0.676148
24,fulladdress7,0.307155,0.329891,0.637046
23,fulladdress3,0.286946,0.310152,0.597098
22,fulladdress2,0.277553,0.301074,0.578627
21,fulladdress1,0.265026,0.287666,0.552691
68,"fulladdress,homephone31",0.228700,0.254851,0.483551
50,"ssn,namedob31",0.227246,0.253519,0.480765
32,namedob31,0.226887,0.253852,0.480739


In [46]:
KS.to_csv('Variables Ranking.csv')